In [1]:
import pandas as pd
import numpy as np
import io
from tqdm import tqdm
from itertools import islice
import langid
import re
from concurrent import futures
import swifter
import asyncio
import nltk
# import vaex
from nltk.tokenize import word_tokenize
from nltk import  pos_tag
from nltk import WordNetLemmatizer
import modin.pandas as pdmd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA


import warnings
warnings.filterwarnings("ignore")

In [11]:
a = pd.read_csv('data.csv')

In [14]:
with open("tfidf_body.pickle", 'rb') as f:
    tfidf_body = pickle.load(f)

# Создание масенького датасета из большого

Нашла я датасет гитхабовских проблем на 2гб

In [8]:
data = pd.read_csv("github_issues.csv")
data

,issue_url,issue_title,body
0,"""https://github.com/zhangyuanwei/node-images/i...",can't load the addon. issue to: https://github...,can't load the addon. issue to: https://github...
1,"""https://github.com/Microsoft/pxt/issues/2543""",hcl accessibility a11yblocking a11ymas mas4.2....,user experience: user who depends on screen re...
2,"""https://github.com/MatisiekPL/Czekolada/issue...",issue 1265: issue 1264: issue 1261: issue 1260...,┆attachments: <a href= https:& x2f;& x2f;githu...
3,"""https://github.com/MatisiekPL/Czekolada/issue...",issue 1266: issue 1263: issue 1262: issue 1259...,gitlo = github x trello\n---\nthis board is no...
4,"""https://github.com/MatisiekPL/Czekolada/issue...",issue 1288: issue 1285: issue 1284: issue 1281...,┆attachments: <a href= https:& x2f;& x2f;githu...
...,...,...,...
5332148,"""https://github.com/bayborodin/ror-full-3/issu...","создать модуль instancecounter, содержащий сле...","методы класса: - instances, который возвращает..."
5332149,"""https://github.com/eclipse/paho.mqtt.java/iss...",at org.eclipse.paho.client.mqttv3.internal.cli...,- bug exists release version 1.1.0 master bran...
5332150,"""https://github.com/rzwitserloot/lombok/issues...",java.lang.linkageerror: loader constraint viol...,java.lang.linkageerror: loader constraint viol...
5332151,"""https://github.com/Gizra/productivity/issues/...",node : pdoexception: sqlstate 40001 : serializ...,view details in rollbar: https://rollbar.com/b...


In [9]:
data.describe()

,issue_url,issue_title,body
count,5332153,5332153,5332153
unique,5315477,4862563,5041808
top,"""https://github.com/cmty-test/cmty-repository-...",first from flow in uk south,first from flow in uk south
freq,297,90133,90133


In [10]:
slice_data = data.sample(frac=1, random_state=42)[:500_000]
slice_data = slice_data.reset_index(drop=True)

In [11]:
slice_data.to_csv("github_issues_slice.csv", index=False)

## Проведу мини-анализ по ускорению обработки данных

Для начала мне нужно выделить из исходного датасета только текст на английском языке (мне так хочется). Для этой задачи уже можно попробовать ускорять

In [20]:
def is_en(text):
    return text if langid.classify(text)[0] == 'en' else pd.NA

In [5]:
def is_en_md(text):
    return text if langid.classify(text)[0] == 'en' else pdmd.NA

### read_csv

In [5]:
# %%timeit

slice_data = pd.read_csv("github_issues_slice.csv")[:1000]

In [6]:
# %%timeit

slice_data_md = pdmd.read_csv("github_issues_slice.csv")[:1000]

### Apply by pandas

In [16]:
%%timeit

res = slice_data.body.apply(is_en)
res.dropna()

8.9 s ± 181 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

res = slice_data_md.body.apply(is_en_md)
res.dropna()

7.18 s ± 139 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### future ThreadPoolExecutor

In [7]:
# %%timeit

with futures.ThreadPoolExecutor() as executor:
    res = pd.Series(executor.map(is_en, slice_data.body))
    res.dropna()

In [8]:
# %%timeit

with futures.ThreadPoolExecutor() as executor:
    res = pdmd.Series(executor.map(is_en_md, slice_data_md.body))
    res.dropna()

### future ProcessPoolExecutor

In [24]:
%%writefile ProcessPoolExecutor_pd.py

import pandas as pd
import langid
import time
from concurrent import futures


def is_en(text):
    return text if langid.classify(text)[0] == 'en' else pd.NA

def main(df):
    start = time.perf_counter()

    with futures.ProcessPoolExecutor() as executor:
        res = pd.Series(executor.map(is_en, df.body))

    print(len(res.dropna()))
    print(time.perf_counter() - start)

if __name__ == '__main__':
    df = pd.read_csv("github_issues_slice.csv")[:1000]

    main(df)

Writing ProcessPoolExecutor_pd.py


In [26]:
!python ProcessPoolExecutor_pd.py

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()

Perhaps you already have a cluster running?
Hosting the HTTP server on port 53227 instead


925
24.9550099
925
19.636662700000002


In [27]:
%%writefile ProcessPoolExecutor_pdmd.py

import pandas as pd
import modin.pandas as pdmd
import langid
import time
from concurrent import futures


def is_en(text):
    return text if langid.classify(text)[0] == 'en' else pd.NA

def main(df):
    start = time.perf_counter()

    with futures.ProcessPoolExecutor() as executor:
        res = pdmd.Series(executor.map(is_en, df.body))

    print(len(res.dropna()))
    print(time.perf_counter() - start)

if __name__ == '__main__':
    df = pdmd.read_csv("github_issues_slice.csv")[:1000]

    main(df)

Writing ProcessPoolExecutor_pdmd.py


In [28]:
!python ProcessPoolExecutor_pdmd.py

### swifter

In [21]:
%%timeit

res = slice_data.body.swifter.apply(is_en)
res.dropna()

Pandas Apply: 100%|██████████| 1000/1000 [00:08<00:00, 111.75it/s]

8.88 s ± 885 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### asyncio

In [24]:
%%writefile asyncio_1.py
import pandas as pd
import langid
import asyncio
import time

async def is_en(df, text):
    if langid.classify(text)[0] == 'en':
        df.loc[df.body == text, 'body'] = text
    else:
        df.loc[df.body == text, 'body'] = pd.NA

async def main(df):
    start = time.perf_counter()
    await asyncio.gather(*[is_en(df, text) for text in df.body])
    print(len(df.dropna()))
    print(time.perf_counter() - start)

if __name__ == '__main__':
    df = pd.read_csv("github_issues_slice.csv")[:1000]

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main(df))
    loop.close()

Overwriting asyncio_1.py


In [25]:
# %%timeit
!python asyncio_1.py

925
17.1797964


### vaex

In [5]:
dv = vaex.from_csv("github_issues_slice.csv", convert=True, chunk_size=1_000_000)

In [6]:
dv = vaex.open('github_issues_slice.csv.hdf5')[:1000]

In [7]:
def is_en(text):
    return np.where(langid.classify(text)[0] == 'en', text, np.nan)
    # return text if langid.classify(text)[0] == 'en' else np.nan

In [8]:
dv.body = dv.body.apply(is_en)

In [9]:
%%time
len(dv.dropna())

Wall time: 27 s


1000

# Result

In [21]:
data = pd.read_csv("github_issues_slice.csv")[:10_000]

In [22]:
with futures.ThreadPoolExecutor() as executor:
    data.body = pd.Series(executor.map(is_en, data.body))

In [23]:
len(data[data.body.isnull()])

684

In [24]:
data = data.dropna()

In [25]:
data

,issue_url,issue_title,body
0,"""https://github.com/atais/angular-eonasdan-dat...",manually entered dates issues,i use format 'yyyy-mm-dd' option and bind ng-m...
1,"""https://github.com/conveyal/analysis-ui/issue...",highlight segment on map when editing speed,when editing the speed of a segment i.e. when ...
2,"""https://github.com/Tat5ato/Phantasmic-Mind/is...",concept art for the otherworld,"in general, the otherworld is craggy and organ..."
3,"""https://github.com/qmlweb/qmlweb/issues/420""",mousearea and touch event in mobile browser,"hello, in the master branch, mousearea doesn't..."
4,"""https://github.com/pybel/pybel/issues/174""","function to drop graph store and edge store, b...",this function should go in the cache manager. ...
...,...,...,...
9994,"""https://github.com/nextcloud/serverinfo/issue...",settings button gone,when i deselected all kinds of logging entries...
9995,"""https://github.com/WhisperSystems/Signal-Andr...",using without gms,"hello, one of my friends have some technical p..."
9997,"""https://github.com/automl/auto-sklearn/issues...",no module named 'configspace.io' during import...,i encounter this exception when trying to impo...
9998,"""https://github.com/gimli-org/gimli/issues/50""",ship self-testing binaries,"all distributed binaries of pygimli windows, c..."


In [26]:
data = data.reset_index(drop=True)

# Обработка данных

In [27]:
data.body[0]

"i use format 'yyyy-mm-dd' option and bind ng-model= formdata.date to my field. now if i enter '2015-03-05aaaaa', the date is displayed as '2015-03-05', but my $scope.formdata.date becomes '2015-03-05aaaaa'. so even if i write error message to user that date is invalid, in input field it looks like totally valid value '2015-03-05'. how do i stop datetimepicker from automatically correcting my dates? this misleads users."

In [28]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\daris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\daris\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\daris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\daris\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Sentence segmentation

In [29]:
def sentence_segmentation(text: str) -> list:
    if re.match(r'[\.!\?;]', text[-1]):
        text = text[:-1]
    return re.split(r'[\.!\?;]\s', text.lower())

## Tokenization

In [30]:
def tokenization(sentences: list) -> list:
    return [re.split(r'[,:(\s\-)]*\s', s) for s in sentences]

## Lemmatization

In [31]:
from nltk.corpus import wordnet as wn

In [32]:
def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wn.ADJ,
        'V': wn.VERB,
        'N': wn.NOUN,
        'R': wn.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wn.NOUN

In [33]:
def lemmatization(sentences: list) -> list:
    sentences_tag  = [pos_tag(s) for s in sentences] # получаем теги слов каждого предложения
    
    lemmatizer = WordNetLemmatizer()
    lemm_sentences = []
    for sent in sentences_tag:
        pos_tagged = [(word, get_wordnet_pos(tag)) for word, tag in sent]
        lemm_sentences.append([lemmatizer.lemmatize(word, tag) for word, tag in pos_tagged])

    return lemm_sentences

## Delete stopwords

In [34]:
stop_words = set(stopwords.words('english')).union({'', ' '})

In [35]:
def del_stopwords(sentences: list) -> list:
    upd_sentences = []
    re_sub = lambda x: re.sub(r"[\+=\t\r\n,;:\*'\"]+","", x)
    union_sentences = lambda x: list(set().union(*x))

    for sent in sentences:
        upd_sentences.append([
            re_sub(word) for word in sent if re_sub(word) not in stop_words and len(word) not in [1, 2]
        ])
    
    return union_sentences(upd_sentences)

## All

In [36]:
def preprocessing_text(text: str) -> list:
    return del_stopwords(lemmatization(tokenization(sentence_segmentation(text))))

In [38]:
with futures.ThreadPoolExecutor() as executor:
    data['words_body'] = pd.Series(executor.map(preprocessing_text, data.body))
    data['words_title'] = pd.Series(executor.map(preprocessing_text, data.issue_title))

## Vectorization

In [39]:
def load_vectors(fname, limit):
  fin = io.open(fname, 'r', encoding = 'utf-8', newline = '\n', errors = 'ignore')
  n, d = map(int, fin.readline().split())
  data = {}
  for line in tqdm(islice(fin, limit), total = limit):
    tokens = line.rstrip().split(' ')
    data[tokens[0]] = np.array(list(map(float, tokens[1:])))
  return data

vecs = load_vectors('crawl-300d-2M.vec', 1_000_000)  

100%|██████████| 1000000/1000000 [03:05<00:00, 5381.33it/s]


In [54]:
def dummy_fun(doc):
    return doc

In [61]:
tfidf_body = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None)
tfidf_body.fit(data.words_body)

tfidf_title = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None)
tfidf_title.fit(data.words_title)

TfidfVectorizer(preprocessor=<function dummy_fun at 0x000001A93D5E28B0>,
                token_pattern=None,
                tokenizer=<function dummy_fun at 0x000001A93D5E28B0>)

In [41]:
def vectorization_body(words: list):
    return tfidf_body.transform([words]).toarray().squeeze()

def vectorization_title(words: list):
    return tfidf_title.transform([words]).toarray().squeeze()

In [42]:
with futures.ThreadPoolExecutor() as executor:
    data['vectors_body'] = pd.Series(executor.map(vectorization_body, data.words_body))
    data['vectors_title'] = pd.Series(executor.map(vectorization_title, data.words_title))

In [43]:
dim = 300
zero = sum(vecs.values()) / len(vecs)

In [44]:
vocab_body = np.zeros((len(tfidf_body.vocabulary_.keys()), dim))
for key in tqdm(tfidf_body.vocabulary_.keys()):
  vocab_body[tfidf_body.vocabulary_[key]] = vecs.get(key, zero)

100%|██████████| 56719/56719 [00:03<00:00, 18649.21it/s]


In [45]:
vocab_title = np.zeros((len(tfidf_title.vocabulary_.keys()), dim))
for key in tqdm(tfidf_title.vocabulary_.keys()):
  vocab_title[tfidf_title.vocabulary_[key]] = vecs.get(key, zero)

100%|██████████| 11228/11228 [00:00<00:00, 92444.16it/s]


In [46]:
data['vectors_body'] = np.array(data.vectors_body.tolist()).dot(vocab_body).tolist()
data['vectors_title'] = np.array(data.vectors_title.tolist()).dot(vocab_title).tolist()

In [47]:
data.to_csv("data.csv", index=False)

In [48]:
import pickle

def save_pickle(obj, file):
    with open(file, 'wb') as f:
        pickle.dump(obj, f)

In [56]:
save_pickle(tfidf_title, "tfidf_title.pickle")
save_pickle(tfidf_body, "tfidf_body.pickle")